In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.getcwd()

'/Users/fyenne/Downloads/booooks/semester5/tf24/tf24_Folder'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from xgboost import XGBRegressor

# Road the data
X_full = pd.read_csv('./home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('./home-data-for-ml-course/test.csv', index_col='Id')
print(X_full.shape, X_test_full.shape)
#$$


(1460, 80) (1459, 79)


In [4]:
X_full = X_full.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
X_test_full = X_test_full.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)

X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [5]:
missing_val_count_by_column = (X_full.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MasVnrType       8
BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64


In [6]:
X_full.LotFrontage.isnull().sum()

0

---
title: this ends now
author: Siming Yan
---

In [7]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                               train_size=0.8, test_size=0.2,
                                                               random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                   X_train_full[cname].dtype == 'object']

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if
                  X_train_full[cname].dtype in ['float64', 'int64']]

In [8]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [2]:
# 1. preprocessor 1
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
preprocessor = make_column_transformer(
    (SimpleImputer(), numerical_cols),
    (OneHotEncoder(), categorical_cols),
)


NameError: name 'numerical_cols' is not defined

In [9]:
# 2. preprocessor 2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [1]:
# # 3. preprocessor 3
# from sklearn.impute import SimpleImputer

# # Imputation
# my_imputer = SimpleImputer()
# imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# # two different method of imputing. above is fitting, below is mean()
# imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
# #--------------------------------------------
# # Imputation removed column names; put them back
# imputed_X_train.columns = X_train.columns
# imputed_X_valid.columns = X_valid.columns

In [2]:
#

---
preprocessor

In [3]:
# preprocessor

In [ ]:
X_train = preprocessor.fit_transform(X_train)

In [2]:
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100
y_valid = y_valid / 100

NameError: name 'preprocessor' is not defined

In [1]:
input_shape = [X_train.shape[1]]

NameError: name 'X_train' is not defined

In [14]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)



model = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape = input_shape),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
  #--------------------------------------------
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1),
])

In [15]:
model.compile(
    optimizer='adam',
    loss='mae',
)

In [16]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=1024,
    epochs=300,
    callbacks=[early_stopping], # put your callbacks in a list
#     verbose=1,  # turn off training log
)



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [4]:
#